In [30]:
from pathlib import Path
from sklearn.feature_selection import SelectKBest, chi2
import openpyxl
import polars as pl
from nltk.tokenize import TweetTokenizer

# HELPERS

In [31]:
def ruta_parquet(nombre_archivo: str) -> Path:
    ruta = f"data/{nombre_archivo}.parquet"
    rutafinal = Path().parent / ruta
    print(f"Ruta final: {rutafinal}")
    return rutafinal

def cargar_parquet(nombre_parquet: str) -> pl.LazyFrame:
    ruta = ruta_parquet(nombre_parquet) 
    return pl.read_parquet(ruta).lazy()

def guardar_parquet(data: pl.LazyFrame | pl.DataFrame, nombre_parquet: str) -> pl.LazyFrame:
    ruta = ruta_parquet(nombre_parquet)
    if isinstance(data, pl.LazyFrame):
        data.sink_parquet(ruta)
        return cargar_parquet(nombre_parquet)
    else:
        data.write_parquet(ruta)
        return cargar_parquet(nombre_parquet)

In [32]:
IMPRIMIR = True

def imprimir(mensaje: str, parametros=None, collect_lazy_frame=False, **kwargs):
    if not IMPRIMIR:
        return
    # Normaliza la fuente de valores: dict explícito o kwargs
    values = {}
    if isinstance(parametros, dict):
        values.update(parametros)
    values.update(kwargs)

    # Soporte opcional para LazyFrame
    if collect_lazy_frame:
        for k, v in list(values.items()):
            if hasattr(v, "collect"):
                values[k] = v.collect()

    print(mensaje.format(**values))

In [33]:

# DEFINICIONES INICIALES
ruta = Path().parent / "data/opiniones.xlsx"
print(f"Ruta del archivo Excel: {ruta} [exists: {ruta.exists()}]")

workbook = openpyxl.load_workbook(ruta, read_only=True)
hojas = workbook.sheetnames
hojas = [hoja.lower().replace(" ", "_") for hoja in hojas]
for hoja in hojas:
    print(f",{hoja}",end="", flush=True)

def obtener_nombre_hoja(sheet_index: int) -> str:
    """Obtiene el nombre de la hoja dado su índice."""
    nombre_hoja = hojas[sheet_index]
    return nombre_hoja


def calificaciones(calificacion:str) -> int:
    """Convierte la calificación de string a int."""
    calificacion_map = {
        "Pésimo": 1,
        "Malo": 2,
        "Regular": 3,
        "Muy bueno": 4,
        "Excelente": 5
    }
    return calificacion_map.get(calificacion, 0)


def tratar_columna(col:str) -> str:
    """Trata el nombre de una columna para que sea válido en polars."""
    col = col.strip().lower().replace(" ", "_").replace("-", "_")
    return col


def añadir_columna_lugar_turistico(iteracion: int) -> pl.LazyFrame:
    df_excel = pl.read_excel(ruta, sheet_id=iteracion+1).lazy()
    nombre_hoja = obtener_nombre_hoja(iteracion)
    
    df_excel = (
        df_excel
        .with_columns([
            pl.lit(nombre_hoja).alias("lugar_turistico"),
            pl.col("Calificación").map_elements(calificaciones).alias("calificacion_numerica"),
        ])
    )

    df_excel = df_excel.rename({col: tratar_columna(col) for col in df_excel.collect_schema().names()})
    return df_excel


def juntar_df(lista_df: list[pl.LazyFrame]) -> pl.LazyFrame:
    nuevo_df_excel = pl.concat(lista_df)
    return nuevo_df_excel




Ruta del archivo Excel: data/opiniones.xlsx [exists: True]
,basilica_colegiata_560,mercado_hidalgo_600,casa_de_diego_rivera_698,universidad_de_guanajuato_900,alhóndiga_930,teatro_juárez_1,010,jardín_de_la_unión_1,134,callejón_del_beso_1,360,monumento_pípila_1,620,museo_de_las_momias_1,650

# Limpiezsa y tratamiento del dataset

In [34]:
lista_df: list[pl.LazyFrame] = []
for i, hoja_n in enumerate(hojas):
    df = añadir_columna_lugar_turistico(i)
    df_p = guardar_parquet(df, hoja_n)
    lista_df.append(df)

#df_excel = juntar_df(lista_df)


# GUARDAR PARQUET

for df in lista_df:
    imprimir(f"{df.collect()['opinión'][:5]}")
    print(f"Numero de opiniones: {df.collect().height:,}")
    MENSAJE = f" Columnas: {df.collect().columns}"
    imprimir(F"{"="*len(MENSAJE)}")
    imprimir(MENSAJE)
    imprimir(F"{"="*len(MENSAJE)}")


Ruta final: data/basilica_colegiata_560.parquet
Ruta final: data/basilica_colegiata_560.parquet
Ruta final: data/mercado_hidalgo_600.parquet
Ruta final: data/mercado_hidalgo_600.parquet
Ruta final: data/casa_de_diego_rivera_698.parquet
Ruta final: data/casa_de_diego_rivera_698.parquet
Ruta final: data/universidad_de_guanajuato_900.parquet
Ruta final: data/universidad_de_guanajuato_900.parquet
Ruta final: data/alhóndiga_930.parquet
Ruta final: data/alhóndiga_930.parquet
Ruta final: data/teatro_juárez_1,010.parquet
Ruta final: data/teatro_juárez_1,010.parquet
Ruta final: data/jardín_de_la_unión_1,134.parquet
Ruta final: data/jardín_de_la_unión_1,134.parquet
Ruta final: data/callejón_del_beso_1,360.parquet
Ruta final: data/callejón_del_beso_1,360.parquet
Ruta final: data/monumento_pípila_1,620.parquet
Ruta final: data/monumento_pípila_1,620.parquet
Ruta final: data/museo_de_las_momias_1,650.parquet
Ruta final: data/museo_de_las_momias_1,650.parquet
shape: (5,)
Series: 'opinión' [str]
[
	"

# CONTRUCCION DEL VOCABULARIO, DISTRIBUCIÓN FRECUENCIAS

In [35]:
class Texto:
    def __init__(self, lazy_frame: pl.LazyFrame, TOP_PALABRAS:int | None = None):
        self.SOS = "<s>"
        self.EOS = "</s>"
        self.UNK = "<unk>"
        self.tokenizer = TweetTokenizer()
        self.lazy_frame = lazy_frame
        self.TOP_PALABRAS = TOP_PALABRAS

        # Guardamos estados
        self._freq_dist: list[tuple[str, int]] = []
        self._final_vocab: set[str] = set()
        self._corpus: list[list[str]] = []
        

    
    def _tokenizar(self, texto: str) -> list[str]:
        tokens = self.tokenizer.tokenize(texto.lower().strip())
        return tokens


    def _tokenizar_data(self) -> pl.LazyFrame:
        df_limpio = (
            self.lazy_frame
            .drop_nulls(subset=["opinión"])
            .with_columns(
                pl.col("opinión")
                .cast(pl.Utf8)
                .str.strip_chars()                                 # recorta espacios
                .str.replace_all(r'^[“”"\'«»\(\)\[\]\s]+', '')      # limpia INICIO
                .str.replace_all(r'[“”"\'«»\(\)\[\]\s]+$', '')      # limpia FINAL
                .alias("opinión_limpia")
            )
            .filter(pl.col("opinión_limpia").str.len_chars() > 0)


            .with_columns(
                pl.col("opinión_limpia")
                .map_elements(self._tokenizar, return_dtype=pl.List(pl.Utf8))
                .alias("opinión_tokenizada")
            )
            .filter(pl.col("opinión_tokenizada").list.len() > 0)
        )
        return df_limpio

    
    def _distribuir_frecuencias(self) -> pl.LazyFrame:
        lf_counts = (
            self._tokenizar_data()
            .select(pl.col("opinión_tokenizada"))
            .explode("opinión_tokenizada")
            .group_by("opinión_tokenizada")
            .len()
            .rename({
                "len": "frecuencia",
                "opinión_tokenizada": "token"
                })
            .sort("frecuencia", descending=True)
        )
        return lf_counts


    

    # ============== VOCABULARIO FINAL ==============
    def _vocabulario_final(self) -> pl.LazyFrame:
        vocab = (
            self._distribuir_frecuencias()
            .select("token")
        )
        return vocab


    
    
    def __build_corpus(self) -> pl.LazyFrame:
        vocab_set = self.vocabulario
        
        PREFIJO_SOS = pl.lit([self.SOS, self.SOS])
        SUFIJO_EOS = pl.lit([self.EOS])
        UNK_LIT = pl.lit(self.UNK)
        VOCAB_LIT = pl.lit(list(vocab_set))

        corpus = (
            self._tokenizar_data()
            .with_row_index("rid")
            .select("rid", "opinión_tokenizada")
            .explode("opinión_tokenizada")
            .with_columns(
                # Aplica el enmascaramiento
                pl.when(pl.col("opinión_tokenizada").is_in(VOCAB_LIT))
                .then(pl.col("opinión_tokenizada"))
                .otherwise(UNK_LIT)
                .alias("tok_mask")
            )
            # Aquí lo que hacemos es juntar todos los tokens (ya enmascarados) en UNA lista por opinión
            .group_by("rid")
            .agg(pl.col("tok_mask").alias("tokens"))
            .with_columns(
                pl.concat_list([PREFIJO_SOS, pl.col("tokens"), SUFIJO_EOS])
                .alias("opinión_trigram")
            )
            .select("opinión_trigram")
        )
        return corpus
    
    
    # ================ PROPIEDADES ==============

    @property
    def freq_dist(self) -> list[tuple[str, int]]:
        if not self._freq_dist:
            df = (
                self._distribuir_frecuencias()
                .limit(self.TOP_PALABRAS)  # Si es None, no limita
                .collect(engine="streaming")
                .rows()
            )
            self._freq_dist = df
            self.id_map = {token: idx for idx, (token, _) in enumerate(self._freq_dist)}
                     
        return self._freq_dist
    
    @property
    def vocabulario(self) -> dict:
        if not self._final_vocab:
            df = (
                self._vocabulario_final()
                .select("token")
                .collect(engine="streaming")
                .to_series()
                .to_list()
            )
            self._final_vocab = set(df)
        return self._final_vocab
    

    @property
    def corpus(self) -> list[list[str]]:
        
        if not self._corpus:
            self._corpus = (
                self.__build_corpus()
                .collect(engine="streaming")
                .get_column("opinión_trigram")
                .to_list()
            )
        return self._corpus
    
    



## Limpieza
Se esta utilizando una limpieza de codigo de manera aislada y preparada, primero insertando token de inicio y de cierre preparando para poder hacer uso del mismo data set
Lista de cosas:
- Token de inicio y fin de oración

In [36]:
textos = []
for df in lista_df:
    texto = Texto(df, TOP_PALABRAS=10)
    textos.append(texto)


In [37]:
for texto in textos:
    print(f"{"="*10} CORPUS {"="*10}")
    corpus = texto.corpus
    print(f"Tamaño del corpus: {len(corpus):,} opiniones")
    for opinion in corpus[:5]:
        print(opinion)

========== CORPUS ==========
Tamaño del corpus: 560 opiniones
['<s>', '<s>', 'basílica', 'muy', 'bien', 'conservada', ',', 'punto', 'central', 'en', 'la', 'ciudad', ',', 'muy', 'linda', 'por', 'dentro', 'vale', 'la', 'pena', 'darse', 'una', 'vuelta', 'para', 'conocer', ',', 'alto', 'significado', 'religioso', 'y', 'arquitectónico', '.', '</s>']
['<s>', '<s>', 'the', 'basilica', '(', 'guanajuato', 'does', 'not', 'have', 'a', 'cathedral', ',', 'it', 'is', 'situated', 'in', 'the', 'nearby', 'city', 'of', 'leon', ')', 'is', 'truly', 'imposing', 'both', 'during', 'the', 'day', 'and', 'when', 'illuminated', 'at', 'night', '.', 'it', 'also', 'has', 'some', 'lovely', 'polychrome', 'sculpture', 'and', 'a', 'great', 'organ', '.', '</s>']
['<s>', '<s>', 'edificio', 'de', 'la', 'iglesia', 'amarilla', 'es', 'un', 'abigarrado', 'y', 'nahu', 'crear', 'una', 'más', 'bella', 'de', 'atocha', '.', 'he', 'venido', 'a', 'cabo', 'mejor', 'en', 'frente', 'de', 'la', 'misma', 'guanajuato', 'es', 'menor', 'cua

In [38]:

for texto in textos:
    print(f"{"="*10} FREQ_DIST {"="*10}")
    freq_dist = texto.freq_dist
    print(f"Tamaño de la distribución de frecuencias: {len(freq_dist):,} tokens")
    print(freq_dist)
    print(texto.id_map.get("donde"))
    # "donde" tiene un id de 93
    print(texto.id_map)

========== FREQ_DIST ==========
Tamaño de la distribución de frecuencias: 10 tokens
[('.', 1244), (',', 1035), ('de', 918), ('la', 751), ('the', 636), ('a', 470), ('y', 431), ('en', 415), ('es', 371), ('el', 323)]
None
{'.': 0, ',': 1, 'de': 2, 'la': 3, 'the': 4, 'a': 5, 'y': 6, 'en': 7, 'es': 8, 'el': 9}
========== FREQ_DIST ==========
Tamaño de la distribución de frecuencias: 10 tokens
[(',', 1542), ('.', 1523), ('de', 914), ('the', 619), ('a', 591), ('y', 567), ('la', 478), ('and', 409), ('que', 398), ('es', 388)]
None
{',': 0, '.': 1, 'de': 2, 'the': 3, 'a': 4, 'y': 5, 'la': 6, 'and': 7, 'que': 8, 'es': 9}
========== FREQ_DIST ==========
Tamaño de la distribución de frecuencias: 10 tokens
[('.', 1979), (',', 1280), ('de', 1135), ('the', 1029), ('of', 732), ('la', 705), ('a', 695), ('and', 571), ('diego', 501), ('y', 495)]
None
{'.': 0, ',': 1, 'de': 2, 'the': 3, 'of': 4, 'la': 5, 'a': 6, 'and': 7, 'diego': 8, 'y': 9}
========== FREQ_DIST ==========
Tamaño de la distribución de frec

In [39]:
for texto in textos:
    print(f"{"="*10} VOCABULARIO {"="*10}")
    vocabulario = texto.vocabulario
    print(f"Tamaño del vocabulario: {len(vocabulario):,}")
    print(vocabulario)

========== VOCABULARIO ==========
Tamaño del vocabulario: 3,939
{'into', 'recién', 'interiors', 'árboles', 'construcciones', 'saint', 'humanity', 'come', 'justo', 'frescos', 'morning', 'visité', 'edifice', 'esperaban', 'before', 'taken', 'ninguna', 'colourful', 'deleitará', 'admira', 'straddles', 'viendo', 'workmanship', 'delante', 'specific', 'b', 'piece', 'testigos', 'bellísmas', 'adquiridos', 'amazingly', 'único', 'varios', 'multiple', 'here', 'mezclado', 'con', 'neoclassical', 'alhondiga', 'mediana', 'desviven', 'creado', 'it', 'agrado', 'impresiona', 'tower', 'detalle', 'vibrante', 'competir', 'cuadro', 'agua', 'algunos', 'ley', 'solo', 'center', 'america', 'practice', 'fact', 'ante', 'hubiera', 'cross', 'interrumpir', 'wealth', 'atracción', 'mantengan', 'ask', 'elegante', 'too', 'improved', 'cheerful', 'exuberante', 'exciting', 'peregrinaje', 'niños', 'roasted', 'protagonista', 'peatonal', 'originally', 'épocas', 'querido', 'barrocas', 'catholic', 'novia', 'tantas', 'admiring', '

# FEATURE SELECTION

In [40]:
class VecorPalabra:
    def __init__(self, texto: Texto, top_k: int = 50):
        self.texto = texto
        self.top_k = top_k
    
    @property # TODO: ESTO VA en lo de bow
    def k_best(self):
        "basilica_colegiata_560,mercado_hidalgo_600,casa_de_diego_rivera_698,universidad_de_guanajuato_900,alhóndiga_930,teatro_juárez_1,010,jardín_de_la_unión_1,134,callejón_del_beso_1,360,monumento_pípila_1,620,museo_de_las_momias_1,650"
        y_train = hojas
        
        
        feature_selector = SelectKBest(score_func=chi2, k=50)
        feature_selector.fit(self._bow_train_frecuencia(), y_train)
        return feature_selector